In [10]:
!pip install langchain_community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.1 MB/s eta 0:00:00


In [11]:
import os, json
from datetime import datetime
from langchain_openai import OpenAI
from langchain import LLMChain, PromptTemplate
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.callbacks import get_openai_callback, StdOutCallbackHandler
from langchain.memory import ConversationBufferMemory

In [14]:
from google.colab import userdata

client = OpenAI(
    base_url=userdata.get('baseurl'),
    api_key=userdata.get('apikey'),
    model='meta-llama/Llama-3.3-70B-Instruct'
)

In [32]:
def quiz_fn(topic: str) -> str:
    tmpl = [
        "Ты эксперт в составлении квизов"
        "Тема: {topic}",
        "Составь вопрос квиза на заданную тему с 4 вариантами ответа."
        "Формат ответа: <q>str<\q>",
        "<ans1>str<\ans1>",
        "<ans2>str<\ans2>",
        "<ans3>str<\ans3>",
        "<ans4>str<\ans4>",
        "<right_ans>int<\right_ans>"
    ]
    tmpl = "\n".join(tmpl)
    prompt = PromptTemplate.from_template(tmpl)
    # Use RunnableSequence instead of LLMChain and invoke instead of run
    chain = prompt | client
    return chain.invoke({"topic": topic})

tools = [
    Tool.from_function(quiz_fn, name="quiz", description="Генерация вопроса квиза"),
]

In [ ]:
memory = ConversationBufferMemory() # Память

callbacks = [StdOutCallbackHandler()] # Callback-handlers: вывод в stdout + сбор метрик

In [36]:
agent = initialize_agent(
    tools,
    client,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    callbacks=callbacks,
    verbose=True,
    max_iterations=10,
    early_stopping_method="force",
    return_intermediate_steps=True,
    agent_kwargs={
        "prefix": (
            "Ты — генератор квизов. Твоя единственная работа — сгенерировать 5 вопросов с помощью quiz() "
            "Тема квиза — это {input}."
        ),
        "suffix": "Выполни quiz() 5 раз. Собери все ответы quiz() в один python список без изменений"
    }
)

In [37]:
def main(topic_name):
    # собираем метрики по токенам и стоимости
    with get_openai_callback() as cb:
        result = agent({"input": topic_name})
    # сохраняем лог
    log = {
        "timestamp": datetime.utcnow().isoformat(),
        "input": topic_name,
        "output": result["output"],
        "metrics": {
            "total_tokens": cb.total_tokens,
            "prompt_tokens": cb.prompt_tokens,
            "completion_tokens": cb.completion_tokens,
            "total_cost_usd": cb.total_cost
        }
    }
    with open("quiz_run_log.json", "a", encoding="utf-8") as f:
        f.write(json.dumps(log, ensure_ascii=False) + "\n")
    return result

In [38]:
res = main('архитектуры llm агентов')



> Entering new AgentExecutor chain...
Question: Эта тема архитектуры llm агентов. Что нужно сделать?
Thought: Нам нужно сгенерировать 5 вопросов с помощью функции quiz() на тему архитектуры llm агентов.
Action: quiz
Action Input: архитектуры llm агентов
Observation:  
Вопрос: Какова основная цель архитектуры LLM (Большая Языковая Модель) агентов?
<ans1>Распознавание и синтез речи
<ans2>Обработка и анализ естественного языка
<ans3>Создание искусственного интеллекта для принятия решений
<ans4>Робототехника и механика
<right_ans>2<­ight_ans>
Thought: Получен первый вопрос, теперь нужно сгенерировать следующий вопрос.
Action: quiz
Action Input: архитектуры llm агентов
Observation:  
<q>В чем заключается основная цель архитектуры LLM агентов?</q>
<ans1>Создание распределенных систем</ans1>
<ans2>Реализация шифрования данных</ans2>
<ans3>Повышение эффективности обработки языковых моделей</ans3>
<ans4>Разработка алгоритмов компрессии</ans4>
<right_ans>3</right_ans>
Thought: Получен второй в

/usr/local/lib/python3.11/dist-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


In [44]:
res['output']

'[\n"Вопрос: Какова основная цель архитектуры LLM (Большая Языковая Модель) агентов?\\n<ans1>Распознавание и синтез речи\\n<ans2>Обработка и анализ естественного языка\\n<ans3>Создание искусственного интеллекта для принятия решений\\n<ans4>Робототехника и механика\\n<right_ans>2<\xadight_ans>", \n"В чем заключается основная цель архитектуры LLM агентов?\\n<ans1>Создание распределенных систем\\n<ans2>Реализация шифрования данных\\n<ans3>Повышение эффективности обработки языковых моделей\\n<ans4>Разработка алгоритмов компрессии\\n<right_ans>3</right_ans>", \n"Что означает аббревиатура LLM в контексте ИИ?\\n<ans1>Линейная логистическая модель\\n<ans2>Large Language'